In [26]:
library(dplyr)
library(raster)
library(terra)
library(sf)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:terra’:

    intersect, union


The following objects are masked from ‘package:raster’:

    intersect, select, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [78]:
#======================================================
# Working with multipart zip files in unix is annoying
#======================================================
# zip -FF gSSURGO_CONUS.zip --out gSSURGO_CONUS_combined_fix.zip
# unzip gSSURGO_CONUS_combined_fix.zip

#========================================================
# Just printing out the layer names from the Geodatabase
#========================================================
layer_names <- st_layers("/data-store/iplant/home/rjramos/gssurgo/gSSURGO_CONUS.gdb")$name
layer_names

#=========================================
# Reading the raster from the geodatabase
#=========================================
r <- rast(
  "/data-store/iplant/home/rjramos/gssurgo/gSSURGO_CONUS.gdb",
  subds = c("gSSURGO_CONUS")
)


#=====================================================
# Reading the raster atribute table from the database
# Unfortunatly, its stored as just a table
#=====================================================
rat <- st_read(
  "/data-store/iplant/home/rjramos/gssurgo/gSSURGO_CONUS.gdb",
  layer = c("Valu1")
)

#======================================================
# Merging the original RAT (categories in terra speak)
# with the provided rat in the geodatabase (Valu1)
#======================================================
levels(r) <- relocate( # value needs to be the first column
  mutate( # value needs to be numeric
    merge( # merge the tables!
      levels(r),
      rat,
      by.x = "MUKEY", by.y = "mukey"
    ),
    value = as.numeric(Value), # value needs to be lower case
    .keep = "unused" # drop the orginal capitilized column
  ),
  value, .before = 1
)

#================================================
# Just checking that the RAT was added correctly
#================================================
cats(r)


[1] "chaashto"           "chconsistence"      "chdesgnsuffix"     
 [4] "chfrags"            "chorizon"           "chpores"           
 [7] "chstruct"           "chstructgrp"        "chtext"            
[10] "chtexture"          "chtexturegrp"       "chtexturemod"      
[13] "chunified"          "cocanopycover"      "cocropyld"         
[16] "codiagfeatures"     "coecoclass"         "coeplants"         
[19] "coerosionacc"       "coforprod"          "coforprodo"        
[22] "cogeomordesc"       "cohydriccriteria"   "cointerp"          
[25] "comonth"            "component"          "copm"              
[28] "copmgrp"            "copwindbreak"       "corestrictions"    
[31] "cosoilmoist"        "cosoiltemp"         "cosurffrags"       
[34] "cosurfmorphgc"      "cosurfmorphhpp"     "cosurfmorphmr"     
[37] "cosurfmorphss"      "cotaxfmmin"         "cotaxmoistcl"      
[40] "cotext"             "cotreestomng"       "cotxfmother"       
[43] "distinterpmd"       "distlegendmd"       "distmd"            
[46] "featdesc"           "laoverlap"          "legend"            
[49] "legendtext"         "mapunit"            "month"             
[52] "muaggatt"           "muaoverlap"         "mucropyld"         
[55] "mutext"             "sacatalog"          "sainterp"          
[58] "sdvalgorithm"       "sdvattribute"       "sdvfolder"         
[61] "sdvfolderattribute" "mdstatdomdet"       "mdstatdommas"      
[64] "mdstatidxdet"       "mdstatidxmas"       "mdstatrshipdet"    
[67] "mdstatrshipmas"     "mdstattabcols"      "mdstattabs"        
[70] "FEATLINE"           "FEATPOINT"          "MULINE"            
[73] "MUPOINT"            "SAPOLYGON"          "MUPOLYGON"         
[76] "DominantComponent"  "Valu1"              "Valu1_1"           
[79] "SDV_Symbology"

Reading layer `Valu1' from data source 
  `/data-store/iplant/home/rjramos/gssurgo/gSSURGO_CONUS.gdb' 
  using driver `OpenFileGDB'


Warning message:
“no simple feature geometries present: returning a data.frame or tbl_df”


value,MUKEY,aws0_5,aws5_20,aws20_50,aws50_100,aws100_150,aws150_999,aws0_20,aws0_30,⋯,nccpi3soy,nccpi3cot,nccpi3sg,nccpi3all,pctearthmc,rootznemc,rootznaws,droughty,pwsl1pomu,musumcpct
<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>
100000,100000,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,100
100001,100001,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,100
1000027,1000027,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,100
100003,100003,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,100
100005,100005,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,100
100007,100007,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,100
100008,100008,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,100
100009,100009,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,100
100010,100010,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,100
